In [1]:

from variables import sourcepath
import os
import pandas as pd
import numpy as np
import etl.dim_author as auth
import etl.dim_journal as jour
import etl.common_functions as cof
import etl.dim_paper as pape
import etl.database as db
import etl.aggregation_paper as agg_pape
from db_credentials import dwh_db_connection_params
import datetime
import re

In [2]:
eng=db.initialize_engine(connection_params=dwh_db_connection_params)

In [3]:
sentences_with_ents, papers_in_dwh=agg_pape.extract_source_data(eng)
aggregated_papers=agg_pape.calc_agg_columns(sentences_with_ents, papers_in_dwh)

In [5]:
aggregated_papers[aggregated_papers.paper_pk==0].to_dict(orient='records')

[{'paper_pk': 0,
  'keywordgroup_pk': 0,
  'authorgroup_pk': 0,
  'journal_pk': 0,
  'year': Timestamp('1678-01-01 00:00:00'),
  'title': 'MISSING',
  'citekey': 'MISSING',
  'abstract': 'MISSING',
  'no_of_pages': 0,
  'article_source_id': 0,
  'model_element': 'numbered hypothesis',
  'level': 'global level',
  'participants': 'individual participant',
  'no_of_participants': 10.0,
  'collection_method': 'qualitative interview',
  'sampling': 'random sampling',
  'analysis_method': 'cluster analysis',
  'sector': 'airline',
  'region': 'America',
  'metric': 'p-value',
  'metric_value': 1.0,
  'conceptual_method': 'design artifact',
  'topic': 'intellectual property',
  'technology': 'Twitter',
  'theory': 'theory of task technology fit',
  'paradigm': 'critical theory',
  'company_type': 'small and mid-size enterprise',
  'validity': 'external validity'}]

In [9]:
ag_pa=aggregated_papers.drop(aggregated_papers[aggregated_papers.paper_pk==0].index)

In [7]:
row0={'paper_pk': 0,
  'keywordgroup_pk': 0,
  'authorgroup_pk': 0,
  'journal_pk': 0,
  'year': pd.to_datetime(1678, format='%Y').normalize(),
  'title': 'MISSING',
  'citekey': 'MISSING',
  'abstract': 'MISSING',
  'no_of_pages': 0,
  'article_source_id': 0,
  'model_element': 'MISSING',
  'level': 'MISSING',
  'participants': 'MISSING',
  'no_of_participants': 0,
  'collection_method': 'MISSING',
  'sampling': 'MISSING',
  'analysis_method': 'MISSING',
  'sector': 'MISSING',
  'region': 'MISSING',
  'metric': 'MISSING',
  'metric_value': 0,
  'conceptual_method': 'MISSING',
  'topic': 'MISSING',
  'technology': 'MISSING',
  'theory': 'MISSING',
  'paradigm': 'MISSING',
  'company_type': 'MISSING',
  'validity': 'MISSING'}

In [15]:
new=pd.concat([ag_pa,pd.DataFrame([row0])], ignore_index=True)

In [16]:
new[new.paper_pk==0]

,paper_pk,keywordgroup_pk,authorgroup_pk,journal_pk,year,title,citekey,abstract,no_of_pages,article_source_id,...,region,metric,metric_value,conceptual_method,topic,technology,theory,paradigm,company_type,validity
214449,0,0,0,0,1678-01-01,MISSING,MISSING,MISSING,0,0,...,MISSING,MISSING,0.0,MISSING,MISSING,MISSING,MISSING,MISSING,MISSING,MISSING


In [3]:
source_para=cof.load_sourcefile('paragraphs.csv')[['para_id', 'article_id','last_section_title', 'last_subsection_title', 'paragraph_type']]
source_para.head()

,para_id,article_id,last_section_title,last_subsection_title,paragraph_type
0,1_0,1,NaN,NaN,PARAGRAPH
1,1_1,1,Abstract,NaN,ABSTRACT
2,1_2,1,Introduction,NaN,HEADER
3,1_3,1,Introduction,NaN,PARAGRAPH
4,1_4,1,Introduction,NaN,PARAGRAPH


In [4]:
papers_in_dwh=db.load_full_table(eng, 'dim_paper')[['paper_pk', 'article_source_id']]
transformed_para=pd.merge(source_para, papers_in_dwh, how='left', left_on='article_id', right_on='article_source_id')#.drop(columns=['article_source_id', 'article_id'])

In [11]:
transformed_para[transformed_para.paper_pk.isna()].article_id.nunique()

6

In [8]:
papers_in_dwh[papers_in_dwh.article_source_id==2081]

,paper_pk,article_source_id


In [9]:
from_papers_source=cof.load_sourcefile('papers_final.csv')
from_papers_source.columns

Index(['article_id', 'citekey', 'journal_akronym', 'journal', 'year',
       'authors', 'title', 'abstract', 'keywords', 'number_of_pages', 'volume',
       'issue', 'publisher', 'place'],
      dtype='object')

In [10]:
from_papers_source[from_papers_source.article_id==2081]

,article_id,citekey,journal_akronym,journal,year,authors,title,abstract,keywords,number_of_pages,volume,issue,publisher,place
2077,2081,flynn_d_an_empirical_study_of_the_validation_p...,ISJ,Information Systems Journal,1994,"Flynn, D.; Warhurst, R.",An empirical study of the validation process w...,Abstract. An empirical investigation into the...,"design process, method, requirements determina...",28.0,4.0,3,NaN,NaN


In [4]:
sql_query='select paper_pk, heading, paragraph_type,  sentence_string, sentence_type, entity_count, entity_label, entity_name from (select sentence_string, sentence_type, paragraph_pk, entity_count, entity_label, entity_name from (select sentence_pk, entity_count, entity_label, entity_name from fact_entity_detection fed inner join dim_entity de on fed.entity_pk = de.entity_pk) as fact_ent_join left join dim_sentence ds on fact_ent_join.sentence_pk=ds.sentence_pk) as sent_ent_join left join dim_paragraph dp on sent_ent_join.paragraph_pk=dp.paragraph_pk'
sentences_with_ents=db.load_df_from_query(eng, sql_query)
papers_in_dwh=db.load_full_table(eng, 'dim_paper')

In [5]:
#model_element
model_element=sentences_with_ents[sentences_with_ents.entity_label=='MODEL_ELEMENT']
me=model_element.groupby(by='paper_pk')[['entity_name']].agg(lambda x: x.mode()[0]).reset_index()
papers_me=pd.merge(papers_in_dwh, me, how='left', on='paper_pk').rename(columns={'entity_name': 'model_element'}).fillna('MISSING')

In [12]:
#level
level=sentences_with_ents[sentences_with_ents.entity_label=='LEVEL']
level=level.apply(lambda r: weigh_entity_by_sentence_type(['ABSTRACT'], r), axis=1)
le=level.groupby(by='paper_pk')[['entity_name']].agg(lambda x: x.explode().mode()[0]).reset_index()
papers_le=pd.merge(papers_me, le, how='left', on='paper_pk').rename(columns={'entity_name': 'level'}).fillna('MISSING')

In [13]:
#participants
participants=sentences_with_ents[sentences_with_ents.entity_label=='PARTICIPANTS']
participants_w=participants.apply(lambda r: weigh_entity_by_heading(heading_pattern=".*data.*|.*participa.*|.*sample.*|.*method.*", row=r, weigh_sentences=True), axis=1)
pa=participants_w.groupby(by='paper_pk')[['entity_name']].agg(lambda x: x.explode().mode()[0]).reset_index()
papers_pa=pd.merge(papers_le, pa, how='left', on='paper_pk').rename(columns={'entity_name': 'participants'}).fillna('MISSING')

In [7]:
def weigh_entity_by_heading (heading_pattern, row, exclude_entities=[], weigh_sentences=False):
    if row['entity_name'] in exclude_entities:
        row['entity_name']=[row['entity_name']]
        if weigh_sentences:
            row['sentence_string']=[row['sentence_string']] * row['entity_count']
    elif bool(re.match(heading_pattern, row['heading'], re.IGNORECASE)):
        row['entity_name']=[row['entity_name']] * 15 * row['entity_count']
        if weigh_sentences:
            row['sentence_string']=[row['sentence_string']] * 15 * row['entity_count']
    else:
        row['entity_name']=[row['entity_name']] * 10 * row['entity_count']
        if weigh_sentences:
            row['sentence_string']=[row['sentence_string']] * 10 * row['entity_count']
    return row

In [114]:
#no_of_participants
ents_agg_pa=pd.merge(participants_w[['paper_pk', 'sentence_string', 'entity_name']], papers_pa[['paper_pk', 'participants']], how='left', on='paper_pk')
cite_sep='START_CITE .* END_CITE' 
ents_agg_pa['participant_number']=ents_agg_pa.apply(lambda row: list(filter(None, [word_to_int(word) for word in [item for sublist in [re.split(cite_sep, sent)[0].split() for sent in row['sentence_string']] for item in sublist]])) if row['entity_name'][0]==row['participants'] else [], axis=1)
nop=ents_agg_pa.groupby(by='paper_pk')[['participant_number']].agg(lambda x: x.explode().mode()).reset_index()
nop['no_of_participants']=nop.participant_number.apply(lambda x: x if isinstance(x, int) else(x[0] if len(x)!=0 else 0)) 
papers_nop=pd.merge(papers_pa, nop[['paper_pk', 'no_of_participants']], how='left', on='paper_pk').fillna(0)

In [115]:
#collection_method
collection_method=sentences_with_ents[sentences_with_ents.entity_label=='COLLECTION_METHOD']
collection_method_w=collection_method.apply(lambda r: weigh_entity_by_heading(heading_pattern=".*data.*|.*collect.*|.*method.*|.*abstract.*|.*approach.*|.*procedure.*|.*design.*|.*experiment.*", row=r, exclude_entities=['data collection method']), axis=1)
cm=collection_method_w.groupby(by='paper_pk')[['entity_name']].agg(lambda x: x.explode().mode()[0]).reset_index()
papers_cm=pd.merge(papers_nop, cm, how='left', on='paper_pk').rename(columns={'entity_name': 'collection_method'}).fillna('MISSING')


In [116]:
#sampling
sampling=sentences_with_ents[sentences_with_ents.entity_label=='SAMPLING']
sampling_w=sampling.apply(lambda r: weigh_entity_by_heading(heading_pattern=".*data.*|.*collect.*|.*method.*|.*abstract.*|.*sampling.*|.*sample.*|.*design.*", row=r, exclude_entities=['sampling']), axis=1)
sa=sampling_w.groupby(by='paper_pk')[['entity_name']].agg(lambda x: x.explode().mode()[0]).reset_index()
papers_sa=pd.merge(papers_cm, sa, how='left', on='paper_pk').rename(columns={'entity_name': 'sampling'}).fillna('MISSING')

In [117]:
#analysis_method
analysis_method=sentences_with_ents[sentences_with_ents.entity_label=='ANALYSIS_METHOD']
analysis_method_w=analysis_method.apply(lambda r: weigh_entity_by_heading(heading_pattern=".*result.*|.*method.*|.*abstract.*|.*analys.*|.*discuss.*|.*conclusion.*", row=r), axis=1)
am=analysis_method_w.groupby(by='paper_pk')[['entity_name']].agg(lambda x: x.explode().mode()[0]).reset_index()
papers_am=pd.merge(papers_sa, am, how='left', on='paper_pk').rename(columns={'entity_name': 'analysis_method'}).fillna('MISSING')

In [118]:
#sector
sector=sentences_with_ents[sentences_with_ents.entity_label=='SECTOR']
sector_w=sector.apply(lambda r: weigh_entity_by_sentence_type(['ABSTRACT'], r), axis=1)
se=sector_w.groupby(by='paper_pk')[['entity_name']].agg(lambda x: x.explode().mode()[0]).reset_index()
papers_se=pd.merge(papers_am, se, how='left', on='paper_pk').rename(columns={'entity_name': 'sector'}).fillna('MISSING')

In [119]:
#region
region=sentences_with_ents[sentences_with_ents.entity_label=='REGION']
region_w=region.apply(lambda r: weigh_entity_by_sentence_type(['ABSTRACT'], r), axis=1)
reg=region_w.groupby(by='paper_pk')[['entity_name']].agg(lambda x: x.explode().mode()[0]).reset_index()
papers_reg=pd.merge(papers_se, reg, how='left', on='paper_pk').rename(columns={'entity_name': 'region'}).fillna('MISSING')

In [120]:
#metric
metric=sentences_with_ents[sentences_with_ents.entity_label=='METRIC']
metric_w=metric.apply(lambda r: weigh_entity_by_heading(heading_pattern=".*result.*|.*method.*|.*abstract.*|.*analys.*|.*discuss.*|.*conclusion.*|.*test.*|.*metric.*", row=r, weigh_sentences=True), axis=1)
me=metric_w.groupby(by='paper_pk')[['entity_name']].agg(lambda x: x.explode().mode()[0]).reset_index()
papers_me=pd.merge(papers_reg, me, how='left', on='paper_pk').rename(columns={'entity_name': 'metric'}).fillna('MISSING')

In [133]:
#metric_value
ents_agg_me=pd.merge(metric_w[['paper_pk', 'sentence_string', 'entity_name']], papers_me[['paper_pk', 'metric']], how='left', on='paper_pk')
ents_agg_me['all_values']=ents_agg_me.apply(lambda row: list(filter(None, [word_to_float(word) for word in [item for sublist in [re.split(cite_sep, sent)[0].split() for sent in row['sentence_string']] for item in sublist]])) if row['entity_name'][0]==row['metric'] else [], axis=1)
mv=ents_agg_me.groupby(by='paper_pk')[['all_values']].agg(lambda x: x.explode().mode()).reset_index()
mv['metric_value']=mv.all_values.apply(lambda x: x if isinstance(x, float) else(x[0] if len(x)!=0 else 0)) 
papers_mv=pd.merge(papers_me, mv[['paper_pk', 'metric_value']], how='left', on='paper_pk').fillna(0)

In [139]:
#conceptual_method
conceptual_method=sentences_with_ents[sentences_with_ents.entity_label=='CONCEPTUAL_METHOD']
conceptual_method_w=conceptual_method.apply(lambda r: weigh_entity_by_heading(heading_pattern=".*result.*|.*method.*|.*abstract.*|.*analys.*|.*discuss.*|.*conclusion.*", row=r), axis=1)
cm=conceptual_method_w.groupby(by='paper_pk')[['entity_name']].agg(lambda x: x.explode().mode()[0]).reset_index()
papers_cm=pd.merge(papers_mv, cm, how='left', on='paper_pk').rename(columns={'entity_name': 'conceptual_method'}).fillna('MISSING')

In [140]:
#topic
topic=sentences_with_ents[sentences_with_ents.entity_label=='TOPIC']
to=topic.groupby(by='paper_pk')[['entity_name']].agg(lambda x: x.mode()[0]).reset_index()
papers_to=pd.merge(papers_cm, to, how='left', on='paper_pk').rename(columns={'entity_name': 'topic'}).fillna('MISSING')

In [143]:
#technology
technology=sentences_with_ents[sentences_with_ents.entity_label=='TECHNOLOGY']
te=technology.groupby(by='paper_pk')[['entity_name']].agg(lambda x: x.mode()[0]).reset_index()
papers_te=pd.merge(papers_to, te, how='left', on='paper_pk').rename(columns={'entity_name': 'technology'}).fillna('MISSING')

In [147]:
#theory
theory=sentences_with_ents[sentences_with_ents.entity_label=='THEORY']
theory_w=theory.apply(lambda r: weigh_entity_by_heading(heading_pattern="    ", row=r, exclude_entities=['theory']), axis=1)
th=theory_w.groupby(by='paper_pk')[['entity_name']].agg(lambda x: x.explode().mode()[0]).reset_index()
papers_th=pd.merge(papers_te, th, how='left', on='paper_pk').rename(columns={'entity_name': 'theory'}).fillna('MISSING')

In [149]:
#paradigm
paradigm=sentences_with_ents[sentences_with_ents.entity_label=='PARADIGM']
para=paradigm.groupby(by='paper_pk')[['entity_name']].agg(lambda x: x.mode()[0]).reset_index()
papers_para=pd.merge(papers_th, para, how='left', on='paper_pk').rename(columns={'entity_name': 'paradigm'}).fillna('MISSING')
#company_type
company_type=sentences_with_ents[sentences_with_ents.entity_label=='COMPANY_TYPE']
ct=company_type.groupby(by='paper_pk')[['entity_name']].agg(lambda x: x.mode()[0]).reset_index()
papers_ct=pd.merge(papers_para, ct, how='left', on='paper_pk').rename(columns={'entity_name': 'company_type'}).fillna('MISSING')
#validity
validity=sentences_with_ents[sentences_with_ents.entity_label=='VALIDITY']
validity_w=validity.apply(lambda r: weigh_entity_by_heading(heading_pattern=".*result.*|.*method.*|.*measure.*|.*valid.*|.*discuss.*|.*conclusion.*", row=r, exclude_entities=['validity']), axis=1)
va=validity_w.groupby(by='paper_pk')[['entity_name']].agg(lambda x: x.explode().mode()[0]).reset_index()
papers_va=pd.merge(papers_ct, va, how='left', on='paper_pk').rename(columns={'entity_name': 'validity'}).fillna('MISSING')

In [157]:
papers_va.head(25)

,paper_pk,keywordgroup_pk,authorgroup_pk,journal_pk,year,title,citekey,abstract,no_of_pages,article_source_id,...,region,metric,metric_value,conceptual_method,topic,technology,theory,paradigm,company_type,validity
0,1,1,1,92002,2021-01-01,Examining interdependence between product user...,(kevin)_yan_j_examining_interdependence_betwee...,Firm-sponsored online user communities have be...,0,1,...,Germany,p-value,0.0010,participatory design,knowledge sharing,online community,social exchange theory,MISSING,MISSING,MISSING
1,2,330,330,92003,1994-01-01,Computer support for strategic organizational ...,a_wilson_f_computer_support_for_strategic_orga...,While information systems continue to be promo...,10,2,...,MISSING,MISSING,0.0000,business process modeling,systems design,computer,theory of administrative behaviour,MISSING,MISSING,MISSING
2,3,350,350,92004,2008-01-01,Essence: facilitating software innovation,aaen_i_essence_facilitating_software_innovatio...,This paper suggests ways to facilitate creativ...,11,3,...,MISSING,MISSING,0.0000,design science,digital innovation,video conference,theory,MISSING,MISSING,MISSING
3,4,389,389,92005,2014-01-01,Symbolic Action Research in Information System...,aakhus_m_symbolic_action_research_in_informati...,An essay is presented as an introduction to th...,14,4,...,Europe,MISSING,0.0000,theory development,systems design,social media,action theory,research paradigm,MISSING,MISSING
4,5,418,418,92006,2014-01-01,Everything counts in large amounts: a critical...,aaltonen_a_everything_counts_in_large_amounts_...,Contemporary digital ecosystems produce vast a...,14,5,...,London,MISSING,0.0000,theory development,digital ecosystem,network infrastructure,actor network theory,critical realism,MISSING,MISSING
5,6,425,425,92007,2011-01-01,Building nation-wide information infrastructur...,aanestad_m_building_nation_wide_information_in...,Initiatives that seek to realize the vision of...,16,6,...,Denmark,MISSING,0.0000,design principle,information systems strategy,electronic health record,theory,MISSING,MISSING,validity
6,21,764,764,92007,2011-01-01,From cacophony to harmony: A case study about ...,abraham_c_from_cacophony_to_harmony_a_case_stu...,The cacophony of criticisms emanating from an ...,21,21,...,United States,MISSING,0.0000,design artifact,systems implementation,health information system,MISSING,positivism,MISSING,MISSING
7,13237,24572,24572,96,1967-01-01,Sociology and Modern Systems Theory,buckley_w_sociology_and_modern_systems_theory_...,MISSING,0,0,...,MISSING,MISSING,0.0000,MISSING,MISSING,MISSING,MISSING,MISSING,MISSING,MISSING
8,7,435,435,21181,2014-01-01,Infrastructuring Work: Building a State-Wide H...,aanestad_m_infrastructuring_work_building_a_st...,Information and communication technologies tha...,12,7,...,India,MISSING,0.0000,design artifact,IT work,information technology infrastructure,MISSING,MISSING,MISSING,MISSING
9,8,498,498,92008,2012-01-01,Metafraud: A Meta-Learning Framework for Detec...,abbasi_a_metafraud_a_meta_learning_framework_f...,Financial fraud can have serious ramifications...,0,8,...,America,false positive rate,1.0000,design artifact,financial performance,business intelligence,MISSING,MISSING,MISSING,MISSING


In [129]:
def word_to_float(word):
    try:
        return (float(word))
    except:
        pass

In [105]:
def word_to_int(word):
    if bool(re.fullmatch("[0-9]+([,.][0-9]{3})*?", word)):
        return(int(re.sub("[,.]", "", word)))
    else:
        pass

In [84]:
#testwords
w1='hello'
w2='b11'
w3='11b'
w4='11' #11
w5='11.0' 
w6='1.000' #1000
w7='11.00'
w8='112.112' #112112
w9='11,0' 
w10='1,000' #1000
w11='11,00'
w12='112,112' #112112
w13='112,112,122' #112112112
w14='112,112,12'


In [10]:
def weigh_entity_by_sentence_type (incresed_weights_types, row):
    if row['sentence_type'] in incresed_weights_types:
        row['entity_name']=[row['entity_name']] * 15  * row['entity_count']
    else:
        row['entity_name']=[row['entity_name']] * 10 * row['entity_count']
    return row
